# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [1]:
#%pip install -r requirements.txt

In [2]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
import nest_asyncio
from ragas.run_config import RunConfig
import tqdm
from module import *

/opt/environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
embbedings = [
    "multilingual_large",
    "baai_large",
    "mxbai_large",
    "baai_small"
]

llm = [
    "llama3",
    "mistral",
    "phi3",
    "gemma"
]

emb_model = embbedings[0]
llm_model = llm[0]

## 2. Carga de datos 

In [4]:
documents_loader = PyPDFDirectoryLoader("./data")
documents = documents_loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 150 documents


In [5]:
chunks = split_documents(documents, chunk_size=512)
print("Number of chunks: ", len(chunks))

Number of chunks:  901


## 3. Creación de la base de datos vectorial

In [6]:
# Load the existing database.
db = Chroma(
    collection_name="test1",
    persist_directory="../database",
    embedding_function=get_embedding_function(emb_model)
)

/opt/environment/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
# Calculate Page IDs.
chunks_with_ids = calculate_chunk_ids(chunks)

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

Number of existing documents in DB: 0


In [8]:
# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

In [9]:
if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    with tqdm.tqdm(total=len(new_chunks)) as pbar:
        for chunk in new_chunks:
            db.add_documents([chunk], ids=[chunk.metadata["id"]])
            pbar.update(1)
    print("Documents added correctly ✅")
else:
    print("✅ No new documents to add")

print(f"{len(documents)} documents added to the database correctly")

👉 Adding new documents: 901


100%|██████████| 901/901 [02:07<00:00,  7.09it/s]

Documents added correctly ✅
150 documents added to the database correctly


## 4. Preguntas de interés

In [10]:
questions_docs_cat = [
    "Quins drets lingüístics reconeix la Constitució Espanyola?",
    "Què estableix la Constitució Espanyola sobre la detenció preventiva i els drets de les persones detingudes?",
    "Quins són els valors superiors de l'ordenament jurídic espanyol segons la Constitució?",
    "Quines condicions específiques es poden aplicar en un contracte indefinit per a persones amb discapacitat?",
    "Com es defineix i s'estableix la durada del període de prova en un contracte indefinit segons el document?",
    "Quins drets de bonificació a la Seguretat Social es mencionen per a contractes indefinits celebrats amb treballadors pertanyents al Sistema Nacional de Garantia Juvenil?",
]

questions_docs_en = [
    "What are the conditions under which an employment contract's probationary period may be extended or terminated?",
    "What are the stipulations for holiday entitlement and the conditions under which holiday can be carried over to the next year?",
    "What are the employer’s rights concerning termination of employment during and after the probationary period?",
    "What are the proposed alternatives for the structure and election of the UK Parliament’s Second Chamber?",
    "What rights are explicitly protected under the proposed Bill of Rights in the document?",
    "How does the document propose constitutional amendments should be enacted or repealed?",
]

questions_docs_es = [
    "¿Qué derechos lingüísticos reconoce la Constitución Española?",
    "¿Qué establece la Constitución Española sobre la detención preventiva y los derechos de las personas detenidas?",
    "¿Cuáles son los valores superiores del ordenamiento jurídico español según la Constitución?",
    "¿Qué condiciones específicas pueden aplicarse en un contrato indefinido para personas con discapacidad?",
    "¿Cómo se define y establece la duración del período de prueba en un contrato indefinido según el documento?",
    "¿Qué derechos de bonificación a la Seguridad Social se mencionan para contratos indefinidos celebrados con trabajadores pertenecientes al Sistema Nacional de Garantía Juvenil?"
]

questions = questions_docs_cat + questions_docs_en + questions_docs_es

In [11]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=3)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

Ahora creamos las respuestas esperadas para cada pregunta.

In [12]:
answers_docs_cat = [
    "La Constitució Espanyola reconeix el castellà com a llengua oficial de l'Estat i estableix que tots els espanyols tenen el deure de conèixer-la i el dret a usar-la. També reconeix que les altres llengües espanyoles seran oficials en les respectives Comunitats Autònomes d'acord amb els seus Estatuts, i protegeix la riquesa de les diferents modalitats lingüístiques d'Espanya com un patrimoni cultural que ha de ser objecte d'especial respecte i protecció.",
    "La Constitució Espanyola estableix que la detenció preventiva no pot durar més del temps estrictament necessari per a la realització de les averiguacions, i en tot cas, el detingut ha de ser posat en llibertat o a disposició de l'autoritat judicial en un termini màxim de 72 hores. A més, garanteix el dret a ser informat de forma immediata i comprensible sobre els drets i les raons de la detenció, així com el dret a l'assistència d'un advocat.",
    "Segons la Constitució Espanyola, els valors superiors de l'ordenament jurídic espanyol són la llibertat, la justícia, la igualtat i el pluralisme polític.",
    "En un contracte indefinit per a persones amb discapacitat, es poden aplicar condicions específiques com bonificacions en la quota empresarial a la Seguretat Social, subvencions, i adaptacions del lloc de treball segons les característiques de la persona. A més, es poden establir períodes de prova i d'adaptació al treball adequats a la situació del treballador.",
    "La durada del període de prova en un contracte indefinit es defineix d'acord amb el que estableix l'Estatut dels Treballadors, respectant les normes legals vigents. Pot variar segons el grup professional o el nivell del treballador, i en alguns casos pot ser de fins a un any si s'acull a certes disposicions legals.",
    "Per als contractes indefinits celebrats amb treballadors pertanyents al Sistema Nacional de Garantia Juvenil, es mencionen bonificacions de 300 euros mensuals en la cotització empresarial a la Seguretat Social durant un període de 6 mesos. En el cas de contractes a temps parcial, la bonificació s'ajusta proporcionalment al percentatge de la jornada.",
]

answers_docs_en = [
    "The probationary period may be extended or terminated if the employee's work performance is not up to the required standard or if the working relationship is not agreeable. Both parties are required to give one week's written notice during this period.",
    "The holiday entitlement is 5.6 weeks per year, pro-rata for part-time employees. Holidays must be taken within the year, and carrying over untaken holidays requires prior agreement, except in cases of sickness preventing the leave.",
    "During the probationary period, both parties must give one week's notice to terminate employment. After the probationary period, four weeks' notice is required, increasing by one week per year of service up to a maximum of twelve weeks. The employer may terminate employment without notice in cases of serious breach, gross misconduct, or gross negligence.",
    "The document offers alternatives for the Second Chamber: a directly elected body representing the nations and regions of the UK, or a Chamber of Experts appointed for 15 years based on recognized expertise. It also suggests the possibility of the Second Chamber having the power to veto bills under certain conditions.",
    "The proposed Bill of Rights includes rights such as the right to life, freedom from torture, slavery, and forced labor, and the right to a fair trial. Other rights include freedom of thought, expression, assembly, and the right to marry and found a family.",
    "The document proposes that constitutional amendments could be enacted or repealed by Parliament through ordinary legislative procedures, with some suggesting the need for a two-thirds majority in both Houses and approval by a public referendum."
]

answers_docs_es = [
    "La Constitución Española reconoce el castellano como lengua oficial del Estado y establece que todos los españoles tienen el deber de conocerla y el derecho a usarla. También reconoce que las demás lenguas españolas serán oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos, y protege la riqueza de las distintas modalidades lingüísticas de España como un patrimonio cultural que debe ser objeto de especial respeto y protección.",
    "La Constitución Española establece que la detención preventiva no puede durar más del tiempo estrictamente necesario para la realización de las averiguaciones, y en todo caso, el detenido debe ser puesto en libertad o a disposición de la autoridad judicial en un plazo máximo de 72 horas. Además, garantiza el derecho a ser informado de forma inmediata y comprensible sobre los derechos y las razones de la detención, así como el derecho a la asistencia de un abogado.",
    "Según la Constitución Española, los valores superiores del ordenamiento jurídico español son la libertad, la justicia, la igualdad y el pluralismo político.",
    "En un contrato indefinido para personas con discapacidad, se pueden aplicar condiciones específicas como bonificaciones en la cuota empresarial a la Seguridad Social, subvenciones, y adaptaciones del puesto de trabajo según las características de la persona. Además, se pueden establecer períodos de prueba y de adaptación al trabajo adecuados a la situación del trabajador.",
    "La duración del período de prueba en un contrato indefinido se define de acuerdo con lo establecido en el Estatuto de los Trabajadores, respetando las normas legales vigentes. Puede variar según el grupo profesional o el nivel del trabajador, y en algunos casos puede ser de hasta un año si se acoge a ciertas disposiciones legales.",
    "Para los contratos indefinidos celebrados con trabajadores pertenecientes al Sistema Nacional de Garantía Juvenil, se mencionan bonificaciones de 300 euros mensuales en la cotización empresarial a la Seguridad Social durante un período de 6 meses. En el caso de contratos a tiempo parcial, la bonificación se ajusta proporcionalmente al porcentaje de la jornada.",
]


answers = answers_docs_cat + answers_docs_en + answers_docs_es

Finalmente creamos el DataFrame para evaluar los modelos.

In [13]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [14]:
embeddings = get_embedding_function(emb_model)
llm = Ollama(model=llm_model)

In [15]:
nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False,
        run_config=RunConfig(
            max_retries=30, # Default is 10
            max_wait=180, # Default is 60
            #max_workers=64 # Default is 16
        )
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating: 100%|██████████| 72/72 [07:33<00:00,  6.29s/it]


                                             question  \
0   Quins drets lingüístics reconeix la Constituci...   
1   Què estableix la Constitució Espanyola sobre l...   
2   Quins són els valors superiors de l'ordenament...   
3   Quines condicions específiques es poden aplica...   
4   Com es defineix i s'estableix la durada del pe...   
5   Quins drets de bonificació a la Seguretat Soci...   
6   What are the conditions under which an employm...   
7   What are the stipulations for holiday entitlem...   
8   What are the employer’s rights concerning term...   
9   What are the proposed alternatives for the str...   
10  What rights are explicitly protected under the...   
11  How does the document propose constitutional a...   
12  ¿Qué derechos lingüísticos reconoce la Constit...   
13  ¿Qué establece la Constitución Española sobre ...   
14  ¿Cuáles son los valores superiores del ordenam...   
15  ¿Qué condiciones específicas pueden aplicarse ...   
16  ¿Cómo se define y establece

In [16]:
df_score.head(10)

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,Quins drets lingüístics reconeix la Constituci...,La Constitució Espanyola reconeix el castellà ...,[La Constitució es fonamenta en la indissolubl...,La Constitució Espanyola reconeix el castellà ...,0.500000,0.833968,1.000000,1.000000
1,Què estableix la Constitució Espanyola sobre l...,La Constitució Espanyola estableix que la dete...,[la societat espanyola i mantindran les conseg...,La Constitució Espanyola estableix que la dete...,1.000000,0.880212,1.000000,1.000000
2,Quins són els valors superiors de l'ordenament...,"Segons la Constitució Espanyola, els valors su...",[Títol Preliminar \n \nArticle 1 \n1. Espanya ...,"Segons la Constitució Espanyola, els valors su...",1.000000,0.894722,1.000000,1.000000
3,Quines condicions específiques es poden aplica...,En un contracte indefinit per a persones amb d...,[Els\n\t\ncentres\n\t\nespecials\n\t\nd’ocupac...,En un contracte indefinit per a persones amb d...,0.333333,0.938415,1.000000,1.000000
4,Com es defineix i s'estableix la durada del pe...,La durada del període de prova en un contracte...,[present\n\t\ncontracte\n\t\nserà\n\t\nINDEFIN...,La durada del període de prova en un contracte...,0.000000,0.909788,1.000000,0.500000
5,Quins drets de bonificació a la Seguretat Soci...,Per als contractes indefinits celebrats amb tr...,[gener.\nSi\n\t\nes\n\t\nreuneixen\n\t\nels\n\...,Per als contractes indefinits celebrats amb tr...,0.333333,0.909273,1.000000,1.000000
6,What are the conditions under which an employm...,The probationary period may be extended or ter...,[de la Empresa ...,The probationary period may be extended or ter...,0.333333,0.966420,0.000000,0.000000
7,What are the stipulations for holiday entitlem...,"The holiday entitlement is 5.6 weeks per year,...",[forward any untaken holiday entitlement to th...,"The holiday entitlement is 5.6 weeks per year,...",0.400000,0.890421,0.000000,0.666667
8,What are the employer’s rights concerning term...,"During the probationary period, both parties m...",[If your work performance is not up to the req...,"During the probationary period, both parties m...",1.000000,0.889508,1.000000,0.750000
9,What are the proposed alternatives for the str...,The document offers alternatives for the Secon...,"[Which of the options in the document, if any,...",The document offers alternatives for the Secon...,0.666667,0.939117,0.583333,0.500000


In [17]:
df_score.to_csv(f"./test1_{llm_model}_{emb_model}.csv", index=False)